In [111]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#  read the dataset
df = pd.read_csv("2012-sat-results.csv")

print(df.info())
print("")

# convert all values to numeric
df["SAT Critical Reading Avg. Score"] = pd.to_numeric(df["SAT Critical Reading Avg. Score"], errors="coerce")
df["SAT Math Avg. Score"] = pd.to_numeric(df["SAT Math Avg. Score"], errors="coerce")
df["SAT Writing Avg. Score"] = pd.to_numeric(df["SAT Writing Avg. Score"], errors="coerce")

# Drop rows with NaN values
df = df.dropna(subset=["SAT Critical Reading Avg. Score", "SAT Math Avg. Score", "SAT Writing Avg. Score"])

print(df.info())
print("")

# population params
mu = df["SAT Writing Avg. Score"].mean()
tao = df["SAT Writing Avg. Score"].sum()
sigmasq = df["SAT Writing Avg. Score"].var(ddof=0)

print(f"The mu is: {mu}")
print(f"The tao is: {tao}")
print(f"The sigma^2 is: {sigmasq}")

print("")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   DBN                              478 non-null    object
 1   SCHOOL NAME                      478 non-null    object
 2   Num of SAT Test Takers           478 non-null    object
 3   SAT Critical Reading Avg. Score  478 non-null    object
 4   SAT Math Avg. Score              478 non-null    object
 5   SAT Writing Avg. Score           478 non-null    object
dtypes: object(6)
memory usage: 22.5+ KB
None

<class 'pandas.core.frame.DataFrame'>
Index: 421 entries, 0 to 477
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              421 non-null    object 
 1   SCHOOL NAME                      421 non-null    object 
 2   Num of SA

In [112]:
df.head()

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0


# 1. Divide your population into strata

In [113]:
# Creating Strata based on Number of SAT Test Takers per school

df["Num of SAT Test Takers"] = pd.to_numeric(df["Num of SAT Test Takers"], errors='coerce')

# strata ranges
bins = [0, 50, 100, 200, 400, df["Num of SAT Test Takers"].max()]
labels = [0, 1, 2, 3, 4]

df["Testers_Stratum"] = pd.cut(df["Num of SAT Test Takers"], bins=bins, labels=labels, include_lowest=True)

testers = {
    "Nh": df.groupby("Testers_Stratum").size().tolist(),
    "sigma_sq_h": df.groupby("Testers_Stratum")["Num of SAT Test Takers"].var(ddof=0).tolist()
}
testers

/var/folders/sd/94f0j_590jlf95lxw1njgbq40000gn/T/ipykernel_66165/3740391049.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  "Nh": df.groupby("Testers_Stratum").size().tolist(),
/var/folders/sd/94f0j_590jlf95lxw1njgbq40000gn/T/ipykernel_66165/3740391049.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  "sigma_sq_h": df.groupby("Testers_Stratum")["Num of SAT Test Takers"].var(ddof=0).tolist()


{'Nh': [148, 173, 50, 26, 24],
 'sigma_sq_h': [191.754930606282,
  166.8447325336631,
  688.8196000000003,
  3789.940828402369,
  46378.74305555556]}

In [114]:
# Creating Strata based on SAT Critical Reading Avg. Score

# Intervals for stratum and labels for the intervals
bins = [350, 375, 400, 425, 450, 515, df["SAT Critical Reading Avg. Score"].max()]
labels = [0, 1, 2, 3, 4, 5]

df["Reading_Stratum"] = pd.cut(df["SAT Critical Reading Avg. Score"], bins=bins, labels=labels, include_lowest=True)

# Grouping by stratum, and calculating variances for each stratum
reading = {
    "Nh": df.groupby("Reading_Stratum").size().tolist(),
    "sigma_sq_h": df.groupby("Reading_Stratum")["SAT Critical Reading Avg. Score"].var(ddof=0).tolist()
}
reading

/var/folders/sd/94f0j_590jlf95lxw1njgbq40000gn/T/ipykernel_66165/2721831635.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  "Nh": df.groupby("Reading_Stratum").size().tolist(),
/var/folders/sd/94f0j_590jlf95lxw1njgbq40000gn/T/ipykernel_66165/2721831635.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  "sigma_sq_h": df.groupby("Reading_Stratum")["SAT Critical Reading Avg. Score"].var(ddof=0).tolist()


{'Nh': [102, 116, 75, 31, 36, 21],
 'sigma_sq_h': [49.47260668973475,
  53.543029131985755,
  51.17582222222231,
  33.33402705515095,
  290.65663580246917,
  2211.9319727891143]}

In [115]:
# Creating Strata based on SAT Math Avg. Score

# Intervals for stratum and labels for the intervals
bins = [350, 375, 400, 425, 450, 515, df["SAT Math Avg. Score"].max()]
labels = [0, 1, 2, 3, 4, 5]

df["Math_Stratum"] = pd.cut(df["SAT Math Avg. Score"], bins=bins, labels=labels, include_lowest=True)

# Grouping by stratum, and calculating variances for each stratum
math = {
    "Nh": df.groupby("Math_Stratum").size().tolist(),
    "sigma_sq_h": df.groupby("Math_Stratum")["SAT Math Avg. Score"].var(ddof=0).tolist()
}
math

/var/folders/sd/94f0j_590jlf95lxw1njgbq40000gn/T/ipykernel_66165/2098692543.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  "Nh": df.groupby("Math_Stratum").size().tolist(),
/var/folders/sd/94f0j_590jlf95lxw1njgbq40000gn/T/ipykernel_66165/2098692543.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  "sigma_sq_h": df.groupby("Math_Stratum")["SAT Math Avg. Score"].var(ddof=0).tolist()


{'Nh': [96, 109, 65, 43, 52, 30],
 'sigma_sq_h': [45.99728732638883,
  52.81440956148471,
  52.962366863905295,
  51.59221200648994,
  280.4715236686394,
  3212.0766666666673]}

# 2. Evaluate Population Stratification

In [116]:
sigma_sq = df["SAT Writing Avg. Score"].var(ddof=0)
N = len(df)
n= 80

Nh = np.array(testers["Nh"])
sigma_sq_h = np.array(testers["sigma_sq_h"])

delta_testers = (N - 1) * sigma_sq - np.sum((Nh - 1) * sigma_sq_h)
delta_testers

188464.96477095643

In [117]:
sigma_sq = df["SAT Writing Avg. Score"].var(ddof=0)
N = len(df)
n= 80

Nh = np.array(reading["Nh"])
sigma_sq_h = np.array(reading["sigma_sq_h"])

delta_reading = (N - 1) * sigma_sq - np.sum((Nh - 1) * sigma_sq_h)
delta_reading

1370209.169962898

In [118]:
sigma_sq = df["SAT Writing Avg. Score"].var(ddof=0)
N = len(df)
n= 80

Nh = np.array(math["Nh"])
sigma_sq_h = np.array(math["sigma_sq_h"])

delta_math = (N - 1) * sigma_sq - np.sum((Nh - 1) * sigma_sq_h)
delta_math

1317477.5710010633

In [119]:
max_delta =  max( max(delta_math, delta_reading), delta_testers )
print(f"Largest delta {max_delta}")

Largest delta 1370209.169962898


#### Use SAT Critical Reading Avg. Score strata from here on

# Sampling Procedures

### 3. Stratified Random Sample with Equal Allocation:
Take stratified random sample with size n (chosen in Report 2) with equal allocation. n_h = n / L

In [120]:
# use reading strata divisions
reading

{'Nh': [102, 116, 75, 31, 36, 21],
 'sigma_sq_h': [49.47260668973475,
  53.543029131985755,
  51.17582222222231,
  33.33402705515095,
  290.65663580246917,
  2211.9319727891143]}

In [121]:
# n chosen in report 2 is 80. 
n = 80
L = len(reading["Nh"])
n_h = int(np.ceil(n / L))
n_h

14

In [122]:
# take samples
stratified_samples = []

for stratum_label in range(L):
    stratum = df[df["Reading_Stratum"] == stratum_label]
    stratified_samples.append(stratum.sample(n=n_h, replace=False))
stratified_sample_df = pd.concat(stratified_samples).reset_index(drop=True)
stratified_sample_df

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Testers_Stratum,Reading_Stratum,Math_Stratum
0,04M495,PARK EAST HIGH SCHOOL,51,373.0,379.0,376.0,1,0,1
1,09X276,LEADERSHIP INSTITUTE,30,361.0,353.0,367.0,0,0,0
2,05M367,ACADEMY FOR SOCIAL ACTION: A COLLEGE BOARD SCHOOL,33,354.0,366.0,342.0,0,0,0
3,08X377,BRONX COMMUNITY HIGH SCHOOL,9,367.0,369.0,376.0,0,0,0
4,06M463,HIGH SCHOOL FOR MEDIA AND COMMUNICATIONS,63,369.0,380.0,349.0,1,0,1
...,...,...,...,...,...,...,...,...,...
79,02M413,SCHOOL OF THE FUTURE HIGH SCHOOL,66,517.0,533.0,515.0,1,5,5
80,25Q525,TOWNSEND HARRIS HIGH SCHOOL,278,621.0,651.0,638.0,3,5,5
81,28Q687,QUEENS HIGH SCHOOL FOR THE SCIENCES AT YORK CO...,121,612.0,660.0,596.0,2,5,5
82,01M696,BARD HIGH SCHOOL EARLY COLLEGE,130,624.0,604.0,628.0,2,5,5


### 4. Parameter Estimation and Variance

a) Estimate your parameter of interest using an unbiased estimator


In [123]:
# mu_st_hat = tau_st_hat / N
# tau_st_hat = sum from 1 to L of (N_h * ybar_h)
# ybar_h = (sum from 1 to n_h of (y_hi)) / n_h

ybar_h = stratified_sample_df.groupby("Reading_Stratum")["SAT Writing Avg. Score"].mean().tolist()
tau_st_hat = sum(reading["Nh"][i] * ybar_h[i] for i in range(len(reading["Nh"])))
mu_st_hat = tau_st_hat / N
mu_st_hat

/var/folders/sd/94f0j_590jlf95lxw1njgbq40000gn/T/ipykernel_66165/3971435269.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  ybar_h = stratified_sample_df.groupby("Reading_Stratum")["SAT Writing Avg. Score"].mean().tolist()


362.798948082796

b) Estimate its variance and provide a confidence interval at the alpha level chosen in Report 2. Use the Satterthwaite formula for adjusted degrees of freedom. 

In [124]:
# variance
# var_hat_mu_st_hat = (1/N^2) * var_hat_tau_st_hat
# var_hat_tau_st_hat = sum from 1 to L ((N_h * (N_h - n_h) * (sigma_h^2 / n_h)))
var_hat_tau_st_hat = sum(
    reading["Nh"][i] * (reading["Nh"][i] - n_h) * (reading["sigma_sq_h"][i] / n_h) 
    for i in range(len(reading["Nh"]))
)
var_hat_mu_st_hat = (1 / N**2) * var_hat_tau_st_hat
var_hat_mu_st_hat


0.7595137664926777

In [125]:
# adjusted degrees of freedom
# d = (sum of 1 to L of (a_h * s_h^2)) 
#      / (sum of 1 to L of (a_h * s_h^2) / (n_h - 1))
# a_h = (N_h * (N_h - n_h)) / n_h
s_h_squared = (
    stratified_sample_df
    .groupby("Reading_Stratum")["SAT Writing Avg. Score"]
    .var(ddof=1)
    .tolist()
)
a_h = [reading["Nh"][i] * (reading["Nh"][i] - n_h) / n_h for i in range(len(reading["Nh"]))]
d = (sum(a_h[i] * s_h_squared[i] for i in range(len(a_h))) / (sum(a_h[i] * s_h_squared[i] 
                                                                  for i in range(len(a_h))) / (n_h - 1)))
d


/var/folders/sd/94f0j_590jlf95lxw1njgbq40000gn/T/ipykernel_66165/4228466091.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("Reading_Stratum")["SAT Writing Avg. Score"]


12.999999999999998

In [126]:
from scipy.stats import t
# Confidence Interval with alpha = .05
# mu_st_hat +- t(alpha/2, d) * sqrt(var_hat_mu_st_hat)
alpha = .05
t_crit = t.ppf(1-(alpha/2), d)
lowerBound = mu_st_hat - t_crit * np.sqrt(var_hat_mu_st_hat)
upperBound = mu_st_hat + t_crit * np.sqrt(var_hat_mu_st_hat)
print(f"95% CI for mu_st_hat is: ({lowerBound}, {upperBound}) \n")

95% CI for mu_st_hat is: (360.916184919068, 364.68171124652406) 



In [127]:
nh_proportional = [
    int(round((n * Nh) / N)) 
    for Nh in reading["Nh"]
]
while sum(nh_proportional) != n:
    if sum(nh_proportional) < n:
        nh_proportional[nh_proportional.index(min(nh_proportional))] += 1
    else:
        nh_proportional[nh_proportional.index(max(nh_proportional))] -= 1
stratified_samples_prop = []
for stratum_label in range(L):
    stratum_data = df[df["Reading_Stratum"] == stratum_label]
    sample_size = nh_proportional[stratum_label]
    
    if sample_size > 0:
        sample = stratum_data.sample(n=sample_size, replace=False)
        stratified_samples_prop.append(sample)
stratified_sample_prop_df = pd.concat(stratified_samples_prop).reset_index(drop=True)

stratified_sample_prop_df


,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,Testers_Stratum,Reading_Stratum,Math_Stratum
0,10X437,FORDHAM HIGH SCHOOL FOR THE ARTS,48,355.0,350.0,372.0,0,0,0
1,02M550,LIBERTY HIGH SCHOOL ACADEMY FOR NEWCOMERS,26,368.0,436.0,352.0,0,0,3
2,09X412,BRONX HIGH SCHOOL OF BUSINESS,34,356.0,376.0,351.0,0,0,1
3,04M409,COALITION SCHOOL FOR SOCIAL CHANGE,48,354.0,357.0,373.0,0,0,0
4,11X249,BRONX HEALTH SCIENCES HIGH SCHOOL,74,370.0,381.0,382.0,1,0,1
...,...,...,...,...,...,...,...,...,...
75,30Q580,BACCALAUREATE SCHOOL FOR GLOBAL EDUCATION,61,524.0,568.0,544.0,1,5,5
76,03M479,BEACON HIGH SCHOOL,261,577.0,575.0,592.0,3,5,5
77,02M416,ELEANOR ROOSEVELT HIGH SCHOOL,127,572.0,594.0,592.0,2,5,5
78,02M412,N.Y.C. LAB SCHOOL FOR COLLABORATIVE STUDIES,114,537.0,590.0,550.0,2,5,5
